In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

# 원문 데이터 불러오기 (Kmeans에서 쪼개진 클러스터별 데이터 셋)
- 원문 = Ward에서 쪼개진 클러스터별 데이터 셋!
- 클러스터에 맞는 원문 데이터 + 토큰화된 데이터가 필요합니다

In [2]:
df=pd.read_csv('KB_Kmeans_cluster0.csv')

#feather로 확장자명 바꾸면 2배 빨라집니다!
df.to_feather("연수_cluster0.ftr")
df_origin= pd.read_feather("연수_cluster0.ftr", columns=None, use_threads=True)
df_origin

,Unnamed: 0,review,Mecab_token,cluster
0,247,중국 배대지 추천좀해주세요 지금 나르다 사용중인데 kg에 원이네요 다른 곳은 아직 ...,"['가격_항공_원산지', '고객', '송금', '추천', '최저', '입출고', '...",0
1,255,일본거주하는데 할수있는 사업이 뭐가있을까요 사업시작한지얼마안되서 사입 문제없고 배송...,"['주변', '이용', '문제', '물품', '요청', '구매', '사업', '거주...",0
2,258,알리바바도 구매대행 되나요 알리바바에서 주문하려는데 배송대행 같은거 되나 궁금해요...,"['사례', '구매', '무역', '그램', '추천', '대행', '홈페이지', '...",0
3,272,인테리어 소품 식기류 수입할려고 합니다 초반에 인테리어 소품 수입하다가 중간에 식기...,"['가격_항공_원산지', '고객', '송금', '최저', '무조건', '증명서', ...",0
4,306,질문 중국 사입시 사업자통관으로 진행하면 관부가세와 세금신고 간이 아니고 일반사업...,"['항공_해운', '납부', '입시', '물류', '구매', '한국', '감사', ...",0
...,...,...,...,...
16965,493345,스마트스토어 즉석식품 안녕하세요 스마트스토어 초보자입니다 제가 아는 공장에서 즉석식...,"['스마트', '문제', '즉석식품', '공장', '물건', '초보', '판매', ...",9
16966,493350,네이버 스마트스토어 명품 스마트스토어에서 명품 반지가 원으로 되어있어서 결제해봤는데...,"['바라', '스마트', '명품', '반지', '결제', '범죄', '물건', '네...",9
16967,493412,스마트스토어 질문드립니다 제가 중고나라에서 제 폰 번호로 스마트스토어 가입 원하시는...,"['스마트', '사기', '가입', '인증', '이익', '만약', '나라', '질...",9
16968,493437,스마트 스토어 하는 중 친구가 구매했다는 걸 알게되어 배송비가 아까운 거 같아서 그...,"['친구', '스마트', '그냥', '구매', '배송', '그러', '방법', '스...",9


## 토큰화
- 공출현빈도 코드에서 사용했었던 전처리 코드 그대로 돌려주세요:)

In [3]:
# 토큰화된 데이터가 필요
from tqdm import tqdm
okt = Okt()
def preprocess_okt(text):
    #text = spacing(text) #띄어쓰기 보정 위에서 했으면 필요없습니다
    pos_words = okt.pos(text, stem=True)
    words = [word for word, tag in pos_words if tag in ['Noun', 'Adjective', 'Verb'] ]

    #여기 stopwords는 이전에 sna노드 만들때 넣었던 단어 그대로 넣어주기
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','아니다',
                 '이렇다','저렇다','도','다','돼다','최고','되다','를','같다',
                 '되었다','오다','가다','으로','자','에','와','한','하다','것',
                 '수','있다','않다','그','이다','이었다', '안','억','이건','정도',
                 '개']
    stopped_words = [w for w in words if not w in stopwords]
    stopped_words=list(set(stopped_words))
    return stopped_words

In [4]:
tqdm.pandas()

df_origin['review_token']=df_origin['review'].progress_apply(preprocess_okt)
df_origin['review_token']= df_origin.review_token


100%|██████████| 16970/16970 [05:46<00:00, 49.02it/s] 


In [8]:
df_origin

,Unnamed: 0,review,Mecab_token,cluster,review_token
0,247,중국 배대지 추천좀해주세요 지금 나르다 사용중인데 kg에 원이네요 다른 곳은 아직 ...,"['가격_항공_원산지', '고객', '송금', '추천', '최저', '입출고', '...",0,"[추천, 전문, 상품, 톡, 일, 부담, 영업, 대해, 검색, 포장, 가능, 배송,..."
1,255,일본거주하는데 할수있는 사업이 뭐가있을까요 사업시작한지얼마안되서 사입 문제없고 배송...,"['주변', '이용', '문제', '물품', '요청', '구매', '사업', '거주...",0,"[얼마, 떠오르다, 리셀, 요, 받다, 제, 지급, 미싱, 쫌, 가전, 배송, 거주..."
2,258,알리바바도 구매대행 되나요 알리바바에서 주문하려는데 배송대행 같은거 되나 궁금해요...,"['사례', '구매', '무역', '그램', '추천', '대행', '홈페이지', '...",0,"[알리바바, 추천, 어떻다, 요, 제, 궁금하다, 진짜, 선정, 배송, 시간, 확인..."
3,272,인테리어 소품 식기류 수입할려고 합니다 초반에 인테리어 소품 수입하다가 중간에 식기...,"['가격_항공_원산지', '고객', '송금', '최저', '무조건', '증명서', ...",0,"[전화, 전문, 상품, 등, 톡, 일, 받다, 부담, 영업, 대해, 검색, 국제, ..."
4,306,질문 중국 사입시 사업자통관으로 진행하면 관부가세와 세금신고 간이 아니고 일반사업...,"['항공_해운', '납부', '입시', '물류', '구매', '한국', '감사', ...",0,"[걸, 등, 신고, 검색, 댓글, 필요하다, 고, 붙여넣기, 보다, 과세, 나오다,..."
...,...,...,...,...,...
16965,493345,스마트스토어 즉석식품 안녕하세요 스마트스토어 초보자입니다 제가 아는 공장에서 즉석식...,"['스마트', '문제', '즉석식품', '공장', '물건', '초보', '판매', ...",9,"[들다, 제, 식품, 공식, 궁금하다, 물건, 스토어, 예, 싼값, 문제, 안녕하다..."
16966,493350,네이버 스마트스토어 명품 스마트스토어에서 명품 반지가 원으로 되어있어서 결제해봤는데...,"['바라', '스마트', '명품', '반지', '결제', '범죄', '물건', '네...",9,"[원, 허위, 범죄, 되어다, 결제, 명품, 반지, 거, 가요, 바라다, 궁금하다,..."
16967,493412,스마트스토어 질문드립니다 제가 중고나라에서 제 폰 번호로 스마트스토어 가입 원하시는...,"['스마트', '사기', '가입', '인증', '이익', '만약', '나라', '질...",9,"[가입, 제, 폰, 받다, 궁금하다, 스토어, 그로, 분, 번호, 보다, 인하다, ..."
16968,493437,스마트 스토어 하는 중 친구가 구매했다는 걸 알게되어 배송비가 아까운 거 같아서 그...,"['친구', '스마트', '그냥', '구매', '배송', '그러', '방법', '스...",9,"[중, 유, 걸, 배송비, 친구, 알, 주다, 아깝다, 되어다, 구매, 건가, 시,..."


In [9]:
#저장
df_origin.to_csv('kmeans_clu0_토큰화된버전.csv', index=False, encoding='utf-8-sig')
df_origin.head()

,Unnamed: 0,review,Mecab_token,cluster,review_token
0,247,중국 배대지 추천좀해주세요 지금 나르다 사용중인데 kg에 원이네요 다른 곳은 아직 ...,"['가격_항공_원산지', '고객', '송금', '추천', '최저', '입출고', '...",0,"[추천, 전문, 상품, 톡, 일, 부담, 영업, 대해, 검색, 포장, 가능, 배송,..."
1,255,일본거주하는데 할수있는 사업이 뭐가있을까요 사업시작한지얼마안되서 사입 문제없고 배송...,"['주변', '이용', '문제', '물품', '요청', '구매', '사업', '거주...",0,"[얼마, 떠오르다, 리셀, 요, 받다, 제, 지급, 미싱, 쫌, 가전, 배송, 거주..."
2,258,알리바바도 구매대행 되나요 알리바바에서 주문하려는데 배송대행 같은거 되나 궁금해요...,"['사례', '구매', '무역', '그램', '추천', '대행', '홈페이지', '...",0,"[알리바바, 추천, 어떻다, 요, 제, 궁금하다, 진짜, 선정, 배송, 시간, 확인..."
3,272,인테리어 소품 식기류 수입할려고 합니다 초반에 인테리어 소품 수입하다가 중간에 식기...,"['가격_항공_원산지', '고객', '송금', '최저', '무조건', '증명서', ...",0,"[전화, 전문, 상품, 등, 톡, 일, 받다, 부담, 영업, 대해, 검색, 국제, ..."
4,306,질문 중국 사입시 사업자통관으로 진행하면 관부가세와 세금신고 간이 아니고 일반사업...,"['항공_해운', '납부', '입시', '물류', '구매', '한국', '감사', ...",0,"[걸, 등, 신고, 검색, 댓글, 필요하다, 고, 붙여넣기, 보다, 과세, 나오다,..."


# Gephi에서 export한 데이터 불러오기
- gephi 모듈성+아이겐벡터 들어있는 데이터 불러오기 (**전체 노드 말고 노드 삭제하고 게피 돌리고 나온 최종 export data 사용!**)
- 이후 모듈성을 고려하여 정의된 세그로 나눠주기

In [10]:
#SNA에서 export한 아이겐벡터
#발견된 세그로 쪼개기
cluster_0=pd.read_excel('연수_cluster0_아이겐벡터.xlsx')

#---------1차 클러스터 -----------------
# 나홀로 관세사
cluster_0_0=cluster_0[cluster_0['Modularity Class']==0]
# 해외송금 걱정인형
cluster_0_2=cluster_0[cluster_0['Modularity Class']==2]


#데이터 확인
display(cluster_0_0)
display(cluster_0_2)

,Id,Label,Interval,Degree,Weighted Degree,Modularity Class,Eigenvector Centrality
0,가격,가격,NaN,2610,5768786,0,0.894899
1,가능,가능,NaN,3633,19385826,0,0.942821
2,가능하다,가능하다,NaN,3487,11341114,0,0.955840
3,검수,검수,NaN,1919,3565180,0,0.812572
4,결제,결제,NaN,3008,9396680,0,0.918226
...,...,...,...,...,...,...,...
124,항상,항상,NaN,798,694370,0,0.530079
125,항시,항시,NaN,1000,1433118,0,0.603558
126,핸들링,핸들링,NaN,601,414346,0,0.437023
127,협의,협의,NaN,1160,1384096,0,0.654068


,Id,Label,Interval,Degree,Weighted Degree,Modularity Class,Eigenvector Centrality
129,개인,개인,NaN,1153,1690416,2,0.832048
130,고객,고객,NaN,2666,6309676,2,0.903123
131,도매,도매,NaN,1833,2155152,2,0.780561
132,등록,등록,NaN,2320,4068036,2,0.859981
133,마진,마진,NaN,902,533928,2,0.557136
...,...,...,...,...,...,...,...
256,해결,해결,NaN,1089,752210,2,0.621308
257,환급,환급,NaN,333,234898,2,0.405187
258,환불,환불,NaN,749,366346,2,0.482551
259,회원,회원,NaN,1120,810726,2,0.615069


# Cluster별 주요 단어 선정
- 수기로 작성
- 여러 마이크로 세그먼트에 나올 것으로 예상되는 키워드 선정 지양
- 원문이 엉뚱한 마이크로 세그에 배정될 가능성 존재

In [11]:
# 수기로 키워드 선정하기
# 10개정도
keywords_0_0=['수수료', '세무사', '자료', '결재', '증빙', '증명서', '과세', '세금', '관세', '부가세', '발행', '계산서', '세관', '세율', '배송비', '관세청', '핸들링', '수입', '검사', '부과']
keywords_0_2=['통장', '인보이스' '발급', '대금', '계좌', '소량', '계약', '송금', '입금', '환급', '통관', '샘플', '은행', '거래', '명의', '서류']



# 주요 단어를 가지고 eigenvector dictionary만들기
- 리스트에 들어있는 단어들에 아이겐벡터+10 가중치

In [12]:
# eigen 벡터에 +10 가중치 주기
#dictionary 만들기

def add_eigenvector_dict(gephi_dataset, cluster_keywords):
    dict_=defaultdict()
    for w,e in zip(gephi_dataset.Label, gephi_dataset['Eigenvector Centrality']):
        if w in cluster_keywords:
            dict_[f'{w}']=e+10
        else:
            dict_[f'{w}']=e

    return dict_

In [13]:
# dictionary 세그별로 만들어주기
dict0=add_eigenvector_dict(cluster_0_0, keywords_0_0)
dict2=add_eigenvector_dict(cluster_0_2, keywords_0_2)
dict2

defaultdict(None,
            {'개인': 0.832047693649753,
             '고객': 0.903122707859072,
             '도매': 0.780561494809369,
             '등록': 0.859980882910538,
             '마진': 0.557135550063521,
             '매출': 0.668232781647595,
             '무료': 0.796550306006913,
             '무역': 0.966368643433663,
             '문제': 0.92195500991267,
             '물류': 0.859227331110095,
             '반품': 0.78222735963096,
             '비싸다': 0.673778122810213,
             '비용': 0.915552433723235,
             '사업자': 0.952915240210938,
             '사이트': 0.862107092069753,
             '상품': 0.941306018204312,
             '서비스': 0.831013484435472,
             '소량': 10.760794152513704,
             '송금': 10.996746868748772,
             '위탁': 0.669298511700874,
             '이용': 0.886016037378809,
             '재고': 0.730261036704351,
             '저렴하다': 0.779141301869663,
             '적용': 0.752001396275905,
             '주문': 0.880488959328983,
             '처리': 0.84174

# 원문에 dictionary 적용하기
1. token화된 원문에 dictionary 단어가 들어가면 eigenvector를 더해준다
2. 단어가 존재하지 않을 경우 0을 더한다
* dict0, dict1 각각을 원문에 적용하여 eigenvector합을 구한다

In [14]:
def eigen_sum(df_origin, cluster_dict, column_name):
    '''
    df_origin: 원문데이터
    cluster_dict: 적용하고 싶은 dictionary
    cluster_name: 결과 값이 dataframe으로 나타나는데 여기서 명명할 열이름
    '''

    fin_score=[]

    for token_list in df_origin.review_token:
        cluster_score=[] # 리뷰 한 행 넘어갈 때마다 점수 리셋
        for token in token_list:
            if token in cluster_dict.keys():
                for word, eigen in cluster_dict.items():
                    if token == word:
                        #print(token)
                        cluster_score.append(eigen)

            else:
                cluster_score.append(0)#아이겐벡터가 없을 경우 0
        #print(cluster_score)

        # eigen합 구하기
        sum_eigen=np.sum(cluster_score) #아이겐벡터 합

        fin_score.append(sum_eigen)

    return pd.DataFrame({column_name: fin_score})

In [15]:
# 열이름에 해당하는 변수입니다! 열이름의 맨끝([-1])은 꼭 번호로 끝내게 만들어주세요!
column_name0='eigen_dict0'
column_name2='eigen_dict2'


#eigen_sum 함수 적용
#eigen_sum 함수의 인수 바꿔주기
cluster_0_0=eigen_sum(df_origin, dict0, column_name0)
cluster_0_2=eigen_sum(df_origin, dict2, column_name2)


#데이터합치기
#df_origin.review, df_origin.review_token 두개는 고정!
total_eigen=pd.concat([df_origin.review, df_origin.review_token,
                       cluster_0_0, cluster_0_2], axis=1)
total_eigen

,review,review_token,eigen_dict0,eigen_dict2
0,중국 배대지 추천좀해주세요 지금 나르다 사용중인데 kg에 원이네요 다른 곳은 아직 ...,"[추천, 전문, 상품, 톡, 일, 부담, 영업, 대해, 검색, 포장, 가능, 배송,...",135.402128,41.458067
1,일본거주하는데 할수있는 사업이 뭐가있을까요 사업시작한지얼마안되서 사입 문제없고 배송...,"[얼마, 떠오르다, 리셀, 요, 받다, 제, 지급, 미싱, 쫌, 가전, 배송, 거주...",18.925903,4.787792
2,알리바바도 구매대행 되나요 알리바바에서 주문하려는데 배송대행 같은거 되나 궁금해요...,"[알리바바, 추천, 어떻다, 요, 제, 궁금하다, 진짜, 선정, 배송, 시간, 확인...",18.636752,4.381847
3,인테리어 소품 식기류 수입할려고 합니다 초반에 인테리어 소품 수입하다가 중간에 식기...,"[전화, 전문, 상품, 등, 톡, 일, 받다, 부담, 영업, 대해, 검색, 국제, ...",134.774259,36.553465
4,질문 중국 사입시 사업자통관으로 진행하면 관부가세와 세금신고 간이 아니고 일반사업...,"[걸, 등, 신고, 검색, 댓글, 필요하다, 고, 붙여넣기, 보다, 과세, 나오다,...",108.431708,21.313244
...,...,...,...,...
16965,스마트스토어 즉석식품 안녕하세요 스마트스토어 초보자입니다 제가 아는 공장에서 즉석식...,"[들다, 제, 식품, 공식, 궁금하다, 물건, 스토어, 예, 싼값, 문제, 안녕하다...",2.172983,1.701096
16966,네이버 스마트스토어 명품 스마트스토어에서 명품 반지가 원으로 되어있어서 결제해봤는데...,"[원, 허위, 범죄, 되어다, 결제, 명품, 반지, 거, 가요, 바라다, 궁금하다,...",1.818688,0.000000
16967,스마트스토어 질문드립니다 제가 중고나라에서 제 폰 번호로 스마트스토어 가입 원하시는...,"[가입, 제, 폰, 받다, 궁금하다, 스토어, 그로, 분, 번호, 보다, 인하다, ...",1.694189,1.342713
16968,스마트 스토어 하는 중 친구가 구매했다는 걸 알게되어 배송비가 아까운 거 같아서 그...,"[중, 유, 걸, 배송비, 친구, 알, 주다, 아깝다, 되어다, 구매, 건가, 시,...",11.796758,0.000000


# 최종 클러스터 설정

### eigen_dict가 모두 0일때 따로 파일에 저장
- 모든 Actor에서 추출된 아이겐벡터를 고려하여 한번 더 원문분리할 예정
- 이후에도 어떤 클러스터에 할당되지 않는 원문은 제거

In [16]:
# dictionary점수 모든 열이 0인 값 vs. 0이 아닌값을 분리
zero_rows = total_eigen[(total_eigen['eigen_dict0'] == 0)&(total_eigen['eigen_dict2'] == 0)]

non_zero_rows = total_eigen[~((total_eigen['eigen_dict0'] == 0)&(total_eigen['eigen_dict2'] == 0))]


max_column = non_zero_rows[['eigen_dict0', 'eigen_dict2']].idxmax(axis=1)
non_zero_rows['final']=[_[-1:] for _ in max_column]



#seg 0
seg0=non_zero_rows[non_zero_rows['final']=='0']
display(seg0)
print(seg0.shape)

#seg2
seg2=non_zero_rows[non_zero_rows['final']=='2']
display(seg2)
print(seg2.shape)


#csv파일로 저장
seg0.to_csv('cluster0_seg0_나홀로관세사.csv', encoding='utf-8-sig')
seg2.to_csv('cluster0_seg2_해외송금걱정인형.csv',encoding='utf-8-sig')

C:\Users\tinac\AppData\Local\Temp\ipykernel_15972\1206437850.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_zero_rows['final']=[_[-1:] for _ in max_column]


,review,review_token,eigen_dict0,eigen_dict2,final
0,중국 배대지 추천좀해주세요 지금 나르다 사용중인데 kg에 원이네요 다른 곳은 아직 ...,"[추천, 전문, 상품, 톡, 일, 부담, 영업, 대해, 검색, 포장, 가능, 배송,...",135.402128,41.458067,0
1,일본거주하는데 할수있는 사업이 뭐가있을까요 사업시작한지얼마안되서 사입 문제없고 배송...,"[얼마, 떠오르다, 리셀, 요, 받다, 제, 지급, 미싱, 쫌, 가전, 배송, 거주...",18.925903,4.787792,0
2,알리바바도 구매대행 되나요 알리바바에서 주문하려는데 배송대행 같은거 되나 궁금해요...,"[알리바바, 추천, 어떻다, 요, 제, 궁금하다, 진짜, 선정, 배송, 시간, 확인...",18.636752,4.381847,0
3,인테리어 소품 식기류 수입할려고 합니다 초반에 인테리어 소품 수입하다가 중간에 식기...,"[전화, 전문, 상품, 등, 톡, 일, 받다, 부담, 영업, 대해, 검색, 국제, ...",134.774259,36.553465,0
4,질문 중국 사입시 사업자통관으로 진행하면 관부가세와 세금신고 간이 아니고 일반사업...,"[걸, 등, 신고, 검색, 댓글, 필요하다, 고, 붙여넣기, 보다, 과세, 나오다,...",108.431708,21.313244,0
...,...,...,...,...,...
16965,스마트스토어 즉석식품 안녕하세요 스마트스토어 초보자입니다 제가 아는 공장에서 즉석식...,"[들다, 제, 식품, 공식, 궁금하다, 물건, 스토어, 예, 싼값, 문제, 안녕하다...",2.172983,1.701096,0
16966,네이버 스마트스토어 명품 스마트스토어에서 명품 반지가 원으로 되어있어서 결제해봤는데...,"[원, 허위, 범죄, 되어다, 결제, 명품, 반지, 거, 가요, 바라다, 궁금하다,...",1.818688,0.000000,0
16967,스마트스토어 질문드립니다 제가 중고나라에서 제 폰 번호로 스마트스토어 가입 원하시는...,"[가입, 제, 폰, 받다, 궁금하다, 스토어, 그로, 분, 번호, 보다, 인하다, ...",1.694189,1.342713,0
16968,스마트 스토어 하는 중 친구가 구매했다는 걸 알게되어 배송비가 아까운 거 같아서 그...,"[중, 유, 걸, 배송비, 친구, 알, 주다, 아깝다, 되어다, 구매, 건가, 시,...",11.796758,0.000000,0


(10344, 5)


,review,review_token,eigen_dict0,eigen_dict2,final
7,수입금액이 소액인데 소량만 수입해서 스마트스토어에 팔아보고 싶은데 구매하고 싶은 제...,"[게시, 재료, 받다, 소량, 하나요, 중국, 스토어, 만들다, 관심, 안되다, 확...",19.505856,23.551528,2
9,도움 부탁드려요 현재 온라인 소품샵 창업 예정자입니다 남대문에서 컵 그릇 커트러리 ...,"[가입, 나이, 등, 상품, 자세하다, 받다, 검색, 댓글, 적지, 분, 나오다, ...",24.554306,28.673721,2
18,알리바바 오류해결 알리바바에서 상품 검색하려고 하는데요 的 求 ...,"[알리바바, 끄다, 걸, 상품, 오류, 경우, 검색, 시, 아시, 분, 보안, 안되...",1.682437,2.219975,2
21,중국 사입궁금해요 한달된 셀린이 입니다 딱 한달만에 첫구매와 두번째 구매는 이루어졌...,"[올리다, 전화, 등, 상품, 셀린, 해도, 부담, 댓글, 샘플, 필요하다, 보고,...",49.814473,53.983106,2
24,다들 결제대행으로 쓰고 계신가요 이번에 구매대행 창업 시작하게 된 초보...,"[올리다, 집필, 걸, 상품, 추천, 견학, 신용장, 등, 일, 받다, 년도, 불다...",63.518017,66.073141,2
...,...,...,...,...,...
16955,스마트스토어 해외직구 배우고 싶습니다 꼭 배워보고 싶습니다 그리고 제대로 한 번 열...,"[및, 등록, 꼭, 이내, 번, 보고, 스토어, 분, 마치다, 해외, 찾다, 직구,...",1.655757,1.723897,2
16957,스마트스토어 질문입니다 스마트스토어를 시작을 했는데 많이 알려진 도매사이트에서 먼저...,"[점, 노하우, 먼저, 소량, 기도, 궁금하다, 스토어, 그렇다, 도매, 입하, 안...",2.262368,12.403463,2
16958,스마트스토어 되팔이 안녕하세요 네이버스마트스토어를 하나 만들까 생각중인사람입니다 제...,"[등, 제, 하나, 셀러, 비싸다, 크로켓, 쉐어, 물건, 스토어, 만들다, 거나,...",0.000000,2.823632,2
16961,내공 스마트스토어 캐릭터 상품 이번에 스마트스토어에서 산리오등 캐릭터 상품을 판매하...,"[내공, 등, 상품, 저작권, 라이센스, 취득, 요, 캐릭터, 산리오, 혹시, 문제...",0.000000,1.863261,2


(6585, 5)


# 어느 클러스터에도 할당되지 못한 리뷰 저장

In [20]:
# zero_rows
z1=zero_rows[['review', 'review_token']]
z1.to_csv('(zero)_clu0.csv', encoding='utf-8-sig')
display(z1)

,review,review_token
10035,광고하려고 하는데요 어떻게 해야 효율이 좋을까요 광고하려고 하는데요 어떻게 해야 ...,"[광고, 좋다, 위메프, 잡화, 애견, 용품, 이베이, 티몬, 알다, 번가, 판매,..."
10198,나이키 제품 한정판 정보는 어디서 유튜브 보다보면 나이키나 아디다스나 리셀 또는 구...,"[나이키, 한정판, 보다, 아디다스, 알, 글, 유튜브, 판매, 어떻다, 파다, 리..."
11887,네이버 블로그 vs 스토어팜 질문이요 처음으로 온라인 창업하는데 명품 빈티지 시계를...,"[네이버, 시계, 겠다, 더, 명품, 스토어, 빈티, 창업, 효율, 파다, 질문, ..."
11976,초보 부업 투잡 전자담배 액상 네이버쇼핑이나 쿠팡에서 판매해보세요,"[네이버, 전자담배, 초보, 해보다, 쿠팡, 투잡, 판매, 액, 부업, 쇼핑]"
12245,스마트스토어 시작한 론코러페이션 입니다 이제막 스마트 스토어를 시작한 론코퍼레이션 ...,"[네이버, 코퍼레이션, 션, 보다, 그날, 저, 검색, 왓, 론, 스마트, 성공하다..."
12360,스마트스토어 개설중 카테고리 설정 뭐로 해야하나요 스마트스토어 개설중인데 판매하려는...,"[팔, 류, 드리다, 카테고리, 설정, 개설, 문의, 뭐, 전부, 요, 어떻다, 스..."
12562,할로윈 제품 공급해주실 수 있는 소싱업체 찾습니다 할로윈을 맞이하여 스토어팜에서 할...,"[해주다, 댓글, 파다, 소품, 싱, 스토어, 업체, 찾다, 구성, 판매, 맞이, ..."
13750,블로그마켓 스마트스토어 시작하기 시작하려면 어떤거부터 해야할까요 관심은 있는데 어디...,"[좋다, 구해, 관심, 물건, 조언, 부탁드리다, 거, 스토어, 마켓, 시작, 부터..."
15248,전자담배 및 액상 온라인 소매업 판매 자격 쿠팡 네이버 자사몰 등등 전자담배와 액상...,"[네이버, 분만, 부탁드리다, 및, 어떻다, 필요하다, 온라인, 많다, 전자담배, ..."
15560,스토어팜 창업 문의 스토어팜 창업 준비중입니다 악세사리 귀걸이 등 를 판매하려고 하...,"[부탁드리다, 홍보, 성의, 귀걸이, 내공, 파다, 막막하다, 문의, 어떻다, 준비..."


# === seg 쪼개기 안 하신 분들은 여기까지만 돌리시면 됩니다 ===

# === seg 쪼개기 안 하신 분(지연/승윤/혜빈)님은 아래도 돌리기 ===

# 2차 클러스터 원문복구

In [ ]:
add_cluster_3=pd.read_excel('[추가]Export_홈가드닝클러만_clu0.xlsx')

#----------추가 2차 클러스터는 홈가드닝 저장된 파일에서 다시 문장 재분배-------------
#원문(2차)
seg3 = seg3.reset_index(drop=True)

# 초보식집사
add_cluster_3_0=add_cluster_3[add_cluster_3['Modularity Class']==0]
# 과일/채소재배
add_cluster_3_1=add_cluster_3[add_cluster_3['Modularity Class']==1]
# 베란다정원
add_cluster_3_2=add_cluster_3[add_cluster_3['Modularity Class']==2]
# 플랜테리어
add_cluster_3_3=add_cluster_3[add_cluster_3['Modularity Class']==3]

#---------데이터 확인-------------------
display(add_cluster_3_0)
display(add_cluster_3_1)
display(add_cluster_3_2)
display(add_cluster_3_3)

,Id,Label,Degree,Weighted Degree,Modularity Class,Eigenvector Centrality
6,가지,가지,737,111654,0,0.709057
8,관리,관리,733,101292,0,0.689906
12,쉬다,쉬다,638,84532,0,0.679617
14,처음,처음,661,91154,0,0.669465
15,어렵다,어렵다,552,69070,0,0.641297
16,들이다,들이다,575,73774,0,0.639269
18,상태,상태,555,69892,0,0.622633
19,시작,시작,503,57406,0,0.616829
20,중요하다,중요하다,492,54786,0,0.606800
21,종류,종류,539,67542,0,0.604329


,Id,Label,Degree,Weighted Degree,Modularity Class,Eigenvector Centrality
1,자라다,자라다,1114,209250,1,0.822728
3,뿌리,뿌리,821,167718,1,0.726596
4,분갈이,분갈이,826,162506,1,0.713875
5,자르다,자르다,785,128618,1,0.709441
11,심다,심다,734,115366,1,0.680050
17,죽다,죽다,586,79290,1,0.635333
22,햇빛,햇빛,531,67652,1,0.604188
26,통풍,통풍,493,59216,1,0.572921
27,꽂다,꽂다,439,46646,1,0.549982
32,피다,피다,426,42752,1,0.529819


,Id,Label,Degree,Weighted Degree,Modularity Class,Eigenvector Centrality
2,베란다,베란다,1247,302982,2,0.816623
13,겨울,겨울,696,100286,2,0.671201
23,정원,정원,616,88370,2,0.591664
48,제라늄,제라늄,487,123358,2,0.483365
50,따다,따다,318,32340,2,0.457905
73,화초,화초,304,25130,2,0.406133
76,가을,가을,258,19418,2,0.400633
91,계절,계절,221,15912,2,0.375559
103,창가,창가,208,15230,2,0.355902
106,꽃대,꽃대,274,35930,2,0.352105


,Id,Label,Degree,Weighted Degree,Modularity Class,Eigenvector Centrality
0,화분,화분,1612,479340,3,0.904739
7,좋아하다,좋아하다,758,106824,3,0.708103
9,나무,나무,789,116190,3,0.684317
10,실내,실내,701,95880,3,0.680624
24,쉽다,쉽다,509,63840,3,0.591395
35,초록,초록,431,49134,3,0.523416
39,야자,야자,439,58710,3,0.512718
43,미니,미니,453,47428,3,0.499972
51,플랜,플랜,386,32758,3,0.457867
52,테리어,테리어,387,33232,3,0.457790


In [ ]:
#키워드
add_keywords_3_0=['처음','어렵다','마르다','살리다','신경','말르다','관리','건강하다','초심자']
add_keywords_3_1=['분갈이','심다','열매','씨앗','가지치기','허브','바질','발아', '배양토', '재배', '상추', '토마토', '오렌지']
add_keywords_3_2=['베란다', '겨울','정원', '가을','계절','추위','남향','따다','창가']
add_keywords_3_3=['플랜', '테리어', '초록','미니', '취미', '행잉', '화분', '쉽다', '모양', '정화']

add_dict0=add_eigenvector_dict(add_cluster_3_0, add_keywords_3_0)
add_dict1=add_eigenvector_dict(add_cluster_3_1, add_keywords_3_1)
add_dict2=add_eigenvector_dict(add_cluster_3_2, add_keywords_3_2)
add_dict3=add_eigenvector_dict(add_cluster_3_3, add_keywords_3_3)

In [ ]:
# 열이름에 해당하는 변수입니다! 열이름의 맨끝([-1])은 꼭 번호로 끝내게 만들어주세요!
column_name0='eigen_dict0'
column_name1='eigen_dict1'
column_name2='eigen_dict2'
column_name3='eigen_dict3'


#eigen_sum 함수 적용
#eigen_sum 함수의 인수 바꿔주기
cluster_3_0=eigen_sum(seg3, add_dict0, column_name0)
cluster_3_1=eigen_sum(seg3, add_dict1, column_name1)
cluster_3_2=eigen_sum(seg3, add_dict2, column_name2)
cluster_3_3=eigen_sum(seg3, add_dict3, column_name3)


#데이터합치기
#df_origin.review, df_origin.review_token 두개는 고정!
total_eigen=pd.concat([seg3.review, seg3.review_token,
                       cluster_3_0,cluster_3_1, cluster_3_2, cluster_3_3], axis=1)
total_eigen

,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,eigen_dict3
0,혼자 살기 적당한 크기가 평이라고 하던데 지금 사는 곳이 평 욕실 주방 분리된 곳 ...,"[실내, 넉넉하다, 평이, 깔다, 분리, 비, 제습기, 버리다, 적당하다, 가전제품...",1.526618,0.000000,10.816623,0.680624
1,새찬데 누가 주차하면서 내차 긁었나봐 개빡치는데 이거 어캐 씨씨티비 확인하고 잡으면...,"[티비, 없다, 나다, 적, 친, 덴져로슬리열받는, 튀다, 알, 뽑다, 긁다, 곳,...",0.000000,0.000000,0.000000,0.000000
2,김직시 년 새로운 업무 화분 키우기 그렇지만 김직시는 화분키우기에 재능이 없는데,"[없다, 직시, 키우다, 새롭다, 업무, 김, 화분, 재능, 년]",0.000000,0.000000,0.000000,10.904739
3,내 본디도 구경와줘 공간분리를 조아합니댜,"[분리, 조아하다, 댜, 공간, 본디, 내, 구경, 주다]",0.000000,0.000000,0.000000,0.000000
4,년 된 베이스기타 케이스 열면 상태 난리났겠지 학생 때 산 건데 다시 치고 싶어서 ...,"[나다, 무섭다, 방치, 먹다, 녹, 금방, 습기, 곰팡이, 두렵다, 거지, 두다,...",0.622633,0.000000,10.816623,0.000000
...,...,...,...,...,...,...
2514,엘지 식기세척기 빌트인 가능할까요 여기에 두고싶은데 걸레받이가 센치정도되는데 잘라야...,"[오븐, 없다, 가전, 많다, 자르다, 선, 요, 짜르, 빌트, 빼다, 충분하다, ...",0.981431,0.709441,0.000000,0.684317
2515,누수관련 주택화재보험 문의드립니다 년정도 된 주택을 구입했는데 밖에 테라스 베란다라...,"[없다, 해달라다, 주택, 방이, 다른, 보험, 입자, 차후, 드리다, 곳, 빼다,...",0.350420,0.000000,10.816623,0.000000
2516,둘중하나는 처분 하려구요 참고로 대 후반입니다 둘다 장농속에 있긴한데 둘다 잘 안들...,"[자다, 나다, 많다, 이로, 선택, 킾, 저, 틀리다, 사르다, 킵, 까페, 하나...",0.709057,0.000000,0.000000,0.000000
2517,첫 인사드려요 늘 알고 있던 레몬테라스인데 이제야 가입을 했어요 잘 부탁드립니다 잘...,"[자다, 인사, 반갑다, 지내다, 테라스, 늘, 가입, 이제야, 첫, 드리다, 보다...",0.000000,0.000000,0.000000,0.000000


In [ ]:
# dictionary점수 모든 열이 0인 값 vs. 0이 아닌값을 분리
zero_rows = total_eigen[(total_eigen['eigen_dict0'] == 0)&(total_eigen['eigen_dict1'] == 0) &
                        (total_eigen['eigen_dict2'] == 0)& (total_eigen['eigen_dict3'] == 0)]
                        #(total_eigen['eigen_dict4'] == 0)& (total_eigen['eigen_dict5'] == 0)&(total_eigen['eigen_dict6'] == 0)]

non_zero_rows = total_eigen[~((total_eigen['eigen_dict0'] == 0)&(total_eigen['eigen_dict1'] == 0) &
                        (total_eigen['eigen_dict2'] == 0)& (total_eigen['eigen_dict3'] == 0))]
                        #(total_eigen['eigen_dict4'] == 0)& (total_eigen['eigen_dict5'] == 0)&(total_eigen['eigen_dict6'] == 0))]


max_column = non_zero_rows[['eigen_dict0', 'eigen_dict1', 'eigen_dict2','eigen_dict3']].idxmax(axis=1)
non_zero_rows['final']=[_[-1:] for _ in max_column]



#seg 0
seg0=non_zero_rows[non_zero_rows['final']=='0']
display(seg0)
print(seg0.shape)

#seg1
seg1=non_zero_rows[non_zero_rows['final']=='1']
display(seg1)
print(seg1.shape)

#seg2
seg2=non_zero_rows[non_zero_rows['final']=='2']
display(seg2)
print(seg2.shape)

#seg3
seg3=non_zero_rows[non_zero_rows['final']=='3']
display(seg3)
print(seg3.shape)


#csv파일로 저장
seg0.to_csv('cluster0_seg0_초보식집사.csv', encoding='utf-8-sig')
seg1.to_csv('cluster0_seg1_채소재배.csv',encoding='utf-8-sig')
seg2.to_csv('cluster0_seg2_베란다정원.csv',encoding='utf-8-sig')
seg3.to_csv('cluster0_seg3_플랜테리어.csv',encoding='utf-8-sig')

/var/folders/sf/g5ffdfm57wv22zjhcdsjt1rm0000gn/T/ipykernel_51313/4091403226.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_zero_rows['final']=[_[-1:] for _ in max_column]


,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,eigen_dict3,final
21,퇴사 일 차 일기 여름에 테니스 시간 치는 사람 오늘은 오전 시부터 시까지 시간 동...,"[없다, 나다, 하니, 게임, 지갑, 운동, 현장, 강력하다, 문도, 가볍다, 별로...",25.142447,0.604188,10.457905,10.334857,0
22,이 올리브 나무는 가망 없어보이죠 물이 적었을까요 몇일 물 흠뻑 줘도 이렇네요 이분...,"[없다, 종류, 실내, 금방, 로즈마리, 올리브, 해, 떨어지다, 햇빛, 살, 안되...",32.315626,12.322240,10.816623,1.364940,0
27,이런거 집에 놔둬도 괜찮을까요 제가 어렸을때부터 허리가 좀 안좋았는데 점점 갈수록 ...,"[없다, 부담, 조절, 강도, 필라테스, 점점, 작다, 해, 관절, 평소, 구, 근...",1.174264,0.490965,0.000000,0.000000,0
29,단독 주택에서살고싶은게 꿈임 아파트싫으네요 옛날 집같은 마당있고 음악과 커피도즐길수...,"[없다, 묻다, 어린시절, 다음, 요, 장소, 꿈임, 바다, 추억, 원하다, 꼭, ...",0.709057,0.000000,0.000000,0.399200,0
36,식세 빌트인으로 설치했는데 걸레받이 빌트인 설치했는데 걸레받이가 짧아 아래가 이만큼...,"[없다, 자르다, 설치다, 복구, 요, 넘다, 빌트, 막다, 원래, 분, 스텐딩, ...",11.440452,0.709441,0.000000,0.684317,0
...,...,...,...,...,...,...,...
2504,인테리어 때문에 셀프로 콘크리트 벽에 못 박아본 여시 있을까 어릴 때 나무에 못박고...,"[때문, 깨지다, 도움, 칼, 뚫다, 나름, 구, 사항, 셀프, 필수, 수평, 완,...",10.669465,0.000000,0.000000,0.684317,0
2507,고양이 눈 밟은 발자국 보고 가세요 더러운 부분은 흐린 눈으로 지나쳐주세용 강아지 ...,"[운, 내려오다, 따뜻하다, 춥다, 고양이, 커엽, 건강하다, 지나치다, 쌓이다, ...",33.391861,0.422795,10.671201,0.708103,0
2509,창가에서 세상에 어쩜 한 폭의 그림같아요 평화로운 고양이나라 감사합니다 평화로운 밤...,"[현상, 따뜻하다, 고양이, 외출, 요, 그냥, 폭, 힐링, 따르다, 곰, 중, 사...",12.056857,0.000000,10.355902,0.708103,0
2514,엘지 식기세척기 빌트인 가능할까요 여기에 두고싶은데 걸레받이가 센치정도되는데 잘라야...,"[오븐, 없다, 가전, 많다, 자르다, 선, 요, 짜르, 빌트, 빼다, 충분하다, ...",0.981431,0.709441,0.000000,0.684317,0


(746, 7)


,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,eigen_dict3,final
5,파키우기 주차 반틈 재배함 처음 키울때 파 하얀색 부분만 남기고 자른다음에 화분에 ...,"[함, 자르다, 부분, 남다, 물다, 햇빛, 다음, 화분, 간격, 곳, 재배, 키우...",10.669465,22.272966,0.000000,10.904739,1
11,식물 추천 이사하면서 식물을 넣고싶은데 저희냥님이 초식냥이시거든요 고양이가 싫어하지...,"[없다, 자다, 고용, 먹다, 자르다, 애플, 종류, 다른, 고양이, 로즈마리, 해...",11.925243,34.677199,0.000000,0.895557,1
17,캣닢 키울 수 있을까요 식물 잘 못키우는 똥손이라 화분 딱 두개있는데 하나는 시들시...,"[나다, 따뜻하다, 고양이, 귀리, 시들다, 요, 키우다, 분, 버리다, 보신, 단...",12.935476,32.974158,0.000000,12.456431,1
35,식물 잘 아시는분 우리집 못나니 어떻게 해야예뻐질까요 저희집에온지 한 년정도 된 몬...,"[자르다, 빵끈, 요, 키우다, 땅, 분, 세용, 옆, 커지다, 정말, 그냥, 잎,...",0.379899,25.104865,0.000000,10.904739,1
47,이 식물 뭘까요 안 쓰던 큰 화분이 있어서 몇년 전 부터 제라늄 대충 심고 키우고 ...,"[요, 뽑다, 분리, 키우다, 분, 긴장, 어찌, 생명력, 잎, 주다, 생명, 못,...",0.000000,23.366044,10.883998,21.670906,1
...,...,...,...,...,...,...,...
2477,인터넷티비 브로드밴드랑 유플러스 어떤게 더 보기좋은가요 가입광고는 사절입니다 저는 ...,"[없다, 지원, 요, 비슷하다, 보신, 허다, 그냥, 보기, 옮기다, 이상하다, 플...",0.981943,10.680050,0.000000,0.000000,1
2485,음식물 쓰레기 걍 대충 버려 음식물 쓰레기 버리는 방법과 원칙을 몰라서 물어보는 게...,"[자다, 음, 나다, 법, 많다, 자르다, 때문, 칼, 대충, 작다, 불투명, 가위...",0.000000,0.709441,0.000000,0.000000,1
2486,부산에 엄마집 재개발하려는 거 같은데 평당 으로 분양할 수 있다고 하던데 부산에서 ...,"[없다, 빡센, 별로, 공시지가, 넘다, 분, 가득, 괜찮다, 안좋다, 어딘데, 쭉...",0.000000,0.370641,0.000000,0.000000,1
2487,일반입양글허락 추가계약서 대지에 뿌리내린 나무보다 든든하고 세대를 초월하는 앤티크처...,"[고양이, 단, 소유, 싸리, 연락, 분, 캣맘캣, 후, 매력, 회, 주다, 게시,...",0.938761,1.104621,0.000000,1.060299,1


(361, 7)


,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,eigen_dict3,final
0,혼자 살기 적당한 크기가 평이라고 하던데 지금 사는 곳이 평 욕실 주방 분리된 곳 ...,"[실내, 넉넉하다, 평이, 깔다, 분리, 비, 제습기, 버리다, 적당하다, 가전제품...",1.526618,0.000000,10.816623,0.680624,2
4,년 된 베이스기타 케이스 열면 상태 난리났겠지 학생 때 산 건데 다시 치고 싶어서 ...,"[나다, 무섭다, 방치, 먹다, 녹, 금방, 습기, 곰팡이, 두렵다, 거지, 두다,...",0.622633,0.000000,10.816623,0.000000,2
6,여시들 서울 오늘처럼 더운 날씨에 베란다에 빨래 말리면 잘 말라 이 정도면 안 습한...,"[자다, 말리, 실내, 덥다, 빨래, 모르다, 서울, 오늘, 시, 말리다, 습하다,...",10.340727,0.000000,10.816623,0.680624,2
8,빨래 개고 있는데 창문 열어달라는거 개빡 뜨거운 물로 씻고 나와서 더우니까 환기 핑...,"[환기, 가족, 하래, 티, 뜨겁다, 핑계, 상전, 내, 내다, 그렇다, 달라, 거...",0.000000,0.000000,10.816623,0.000000,2
13,이거 무슨 꽃일까요 선물 받았는데 고냥이한테 해되는 식물인지 몰라서 부탁드려요 난 ...,"[공중, 종류, 시간, 운동, 공, 해, 안심, 호, 시들다, 보여주다, 접다, 양...",1.718910,2.044545,21.274528,0.000000,2
...,...,...,...,...,...,...,...
2510,이거 두개 사줘야 안싸우겠죠 지금 베란다에 집 모형 으로 층 테라스 딸린 집 놓아줬...,"[강도, 원일, 해외, 사주면, 요, 밀리다, 갈아타다, 수정, 버리다, 얼마, 취...",1.348326,0.000000,10.816623,0.000000,2
2511,테라스 길냥이 넋두리 작년 겨울에 태어난 길냥이가 있어요 테라스에 자주 놀러오던 삼...,"[없다, 치즈, 담, 넉넉하다, 고양이, 완전, 바라다, 요, 넘다, 턱, 스티로폼...",1.776637,0.000000,21.129106,0.000000,2
2513,삼성 키친핏이나 빌트인 하신분계신가요 사진들처럼 도어 냉장고 하나랑 김치냉장고 넣을...,"[나다, 고추, 스포크, 튀다, 요, 빌트, 비, 키우다, 욕, 기능, 분, 옆, ...",0.000000,0.490965,10.816623,0.000000,2
2515,누수관련 주택화재보험 문의드립니다 년정도 된 주택을 구입했는데 밖에 테라스 베란다라...,"[없다, 해달라다, 주택, 방이, 다른, 보험, 입자, 차후, 드리다, 곳, 빼다,...",0.350420,0.000000,10.816623,0.000000,2


(739, 7)


,review,review_token,eigen_dict0,eigen_dict1,eigen_dict2,eigen_dict3,final
2,김직시 년 새로운 업무 화분 키우기 그렇지만 김직시는 화분키우기에 재능이 없는데,"[없다, 직시, 키우다, 새롭다, 업무, 김, 화분, 재능, 년]",0.000000,0.000000,0.000000,10.904739,3
9,소소한 일상 초 딸의 지극정성 식물키우기 영심이 제가 베란다가 있는 구축 아파트로 ...,"[일어나다, 자르다, 아지트, 쏟다, 바라다, 요, 어른, 키우다, 뿌듯하다, 보이...",1.159808,22.806817,21.814420,23.031216,3
10,베란다정원 우리집에 맞는 식물 화분 고르기 베란다 정원 부부 취미 추천 하얀미소유정...,"[건강하다, 확실하다, 점심, 추위, 차, 갖다, 후, 해드리다, 괜찮다, 나가다,...",41.200497,48.327759,42.905661,86.814753,3
12,냥이에게 훈육은 의미 없지요 잘 알면서도 오늘은 냥이에게 섭섭한 맘에 안방 문을 첨...,"[없다, 이내, 구르다, 조용하다, 손, 달리, 에혀, 재롱둥이, 쫌, 루틴, 턱,...",0.000000,0.000000,0.000000,0.708103,3
15,고양이에게 안전한 꽃 식물 알려주세요 튤립은 절대 안된다고 안전한거 알려주세요 꽃 ...,"[종류, 뜨다, 철쭉, 고양이, 장미, 사실, 올리브, 안되다, 알, 대표, 저, ...",0.604329,0.000000,0.483365,1.197034,3
...,...,...,...,...,...,...,...
2476,진짜 낮져밤이 만나보신분잇으새요 사람이 낮에는 조용순둥하다가 밤에는 짐승처럼 변하나...,"[자다, 없다, 짐승, 운동선수, 지다, 걷다, 이불, 신랑, 가리다, 네, 다가,...",0.435135,0.000000,0.000000,0.708103,3
2478,강아지를 키우고싶은데 경험이 없어서요 조언 평소 동물을 좋아하긴 하는데 직접적으로 ...,"[없다, 등산, 가장, 운동, 부모님, 평소, 병아리, 강아지, 견종, 항상, 입양...",0.606800,0.000000,0.000000,0.708103,3
2479,댕치원 다니는 강아지 있니 울집 강아지 체력 너어무 좋아서 유치원 말고 산책교육을 ...,"[나다, 쉿, 고양이, 거가, 타, 달리다, 완전, 내내, 티, 넘다, 체력, 어질...",0.000000,0.635333,0.000000,0.708103,3
2505,이사와서 가구 새로 사려고 배치중인데 함 골라주라 방에서 간식 야식 밥 먹거나 혹시...,"[함, 매듭, 분위기, 안락하다, 먹다, 나다, 뜨다, 혹시, 밥, 누, 사려, 간...",0.679617,0.000000,0.000000,11.019174,3


(588, 7)


In [ ]:
z=zero_rows[['review', 'review_token']]
z.to_excel('(2차)zero_실내_clu0.xlsx', encoding='utf-8-sig')

# 최종 할당X 리뷰들
zero_final = pd.concat([z1, z], axis=0)
zero_final.to_excel('zero_실내_clu0.xlsx', encoding='utf-8-sig')
display(zero_final)

,review,review_token
1,새찬데 누가 주차하면서 내차 긁었나봐 개빡치는데 이거 어캐 씨씨티비 확인하고 잡으면...,"[티비, 없다, 나다, 적, 친, 덴져로슬리열받는, 튀다, 알, 뽑다, 긁다, 곳,..."
3,내 본디도 구경와줘 공간분리를 조아합니댜,"[분리, 조아하다, 댜, 공간, 본디, 내, 구경, 주다]"
7,나 이거 수면장애 맞지 새벽 시에 일어난다고 치면 밤 시에 자려고 눕잖아 잠을 못 ...,"[자다, 일어나다, 때문, 시간, 수면장애, 놀라다, 압박, 듯, 치면, 뭔가, 엉..."
73,어제이사왔는대 이거 보일러 수온문제야 온수나 보일러 틀면 보일러에서 우우웅거리면서 ...,"[자다, 면서, 조절, 함, 사오다, 펑, 뜨겁다, 냉수, 바꾸다, 수리, 밸브, ..."
75,맘충들 호들갑 줘 패고싶네 하자보수 하는거 냄새 때문에 페인트 칠부터 먼저해야된다 ...,"[때문, 칠, 우리, 퍼티, 작업, 가능하다, 주다, 임, 빼애, 하자, 호들갑, ..."
...,...,...
2502,함께한지 일 육아일기 육묘일기 고양이와 자녀 조카가 함께하는 휴먼 베이비 키즈 글만...,"[자다, 자녀, 시간, 주택, 고양이, 베스트, 휴먼, 걸리다, 끄다, 집요하다, ..."
2503,쉐어하우스 잘 구해집니까,"[자다, 집, 쉐어, 하우스, 구해]"
2506,다묘가정 집도 이렇게 향긋할 수 있습니다 화장실 냄새 이제 안녕 이모미 삼초니 애들...,"[똥, 가정, 환기, 집도, 지옹, 우리, 탈취, 퓨저, 옹, 덩어리, 가득하다, ..."
2512,정수기 빌트인 정수기 빌트인 혜택 많이 주시는 분 연락 문의주셔서 쪽지로 연락드렸습...,"[지원, 정수기, 월, 연락, 분, 최대, 문의, 현금, 쪽지, 드리다, 주시, 주..."
